# OR Final Project - Code

Marjan Boloori - 810195384 \\
Parmida Valiahdi - 810196586 \\
Kasra Borazjani - 810196662

## Part 0 - Importing the Libraries
First, we import the needed libraries:

In [1]:
import numpy as np
import pandas as pd
import math
import pulp as p

## Part 1 - Loading the Data:

Then, we read the .csv file we have our data inside:

In [2]:
data = pd.read_csv('data3.csv', header = None)
data_array = np.array(data)

We define our "edges" as the existing routes between a powerplant and a city:

In [3]:
edge = np.zeros((10,8))
for i in range(10):
  for j in range(8):
    edge[i][j] = data_array[i+2][j+3]

We load the distances gathered in data collection phase via the code below:

In [4]:
dist = np.zeros((10,8))
for i in range(10):
  for j in range(8):
    dist[i][j] = data_array[i+2][j+11]

The threshold for the source output is read via this code:

In [5]:
sthershold = np.zeros((1,8))
for i in range(14,22):
  sthershold[0][i-14] = float(data_array[i][3]) 

Here, we define the amount of electrical power need that should be fulfilled for each city:

In [6]:
dthershold = np.zeros((1,10))
for i in range(1,11):
  dthershold[0][i-1] = float(data_array[i][19])

We just remove some bugs in our data (turning NANs into zeros):

In [7]:
for i in range(10):
  for j in range(8):
    if math.isnan(dist[i][j]):
      dist[i][j] = 0

## Part 2 - Defining the Problem:

### 2.1. The Problem Itself
Defining our problem as a minimization problem:

In [8]:
Lp_prob = p.LpProblem('Problem', p.LpMinimize)

### 2.2. Variables
Defining variables $x_{ij}$, $S_i$, and $D_j$ for our problem:

In [9]:
for i in range(1,11):
  for j in range(1,9):
    globals()["x{}{}".format(i,j)] = p.LpVariable("x{}{}".format(i,j), 0, None, p.LpContinuous)

for i in range(1,9):
  globals()["S{}".format(i)] = p.LpVariable("S{}".format(i), 0, None, p.LpContinuous)

for i in range(1,11):
  globals()["D{}".format(i)] = p.LpVariable("D{}".format(i), 0, None, p.LpContinuous)

### 2.3. Objective
Defining the objective as the multiplication result of each $x_{ij}$ in each $c_{ij}$ which is recieved from the total distance between source $i$ and demand $j$:

In [10]:
sumJ = None

for i in range(10):
  for j in range(8):
    sumJ += globals()["x{}{}".format(i+1,j+1)]*np.multiply(edge,dist)[i][j]

Lp_prob += sumJ

### 2.4. Constraints

First, we should pay attention to the fact that the sum of all $x_{ij}$ for a specific $i$ gives the output power for source $S_i$:

In [11]:
sumS = None

for i in range(8):
    
  sumx = None

  for j in range(10):
    sumx += globals()["x{}{}".format(j+1,i+1)]*edge[j][i]
    
  Lp_prob += (globals()["S{}".format(i+1)] == sumx)

Also, sum of all $x_{ij}$ for a specific $j$ should hand over the input power for demanding city $D_j$:

In [12]:
sumD = None

for i in range(9):
    
  sumx = None

  for j in range(8):
    sumx += globals()["x{}{}".format(i+1,j+1)]*edge[i][j]
    
  Lp_prob += (globals()["D{}".format(i+1)] == sumx)

The underlying constraint is the fact that sum of all outputs should be equal to the sum of all inputs:

In [13]:
for i in range(8):
  sumS += globals()["S{}".format(i+1)]

for i in range(10):
  sumD += globals()["D{}".format(i+1)]

Lp_prob += (sumS == sumD)

Also, we should implement the fact that each powerplant has a maximum power that it can generate, and each city should at least get the average net used power from the sum of all of its inputs:

In [14]:
for i in range(8):
  Lp_prob += globals()["S{}".format(i+1)] <= sthershold[0][i]

for i in range(10):
  Lp_prob += globals()["D{}".format(i+1)] >= dthershold[0][i]

## Part 3 - Testing the Fomulation and the Program

Having our problem defined, all we need to do is to run the program:

In [15]:
print(Lp_prob)   
status = Lp_prob.solve()   # Solver 
print(p.LpStatus[status])   # The solution status 
# Printing the final solution 
for x in Lp_prob.variables():
    print(x.name, "=", x.varValue)
print('Joptimal = ' + str(p.value(Lp_prob.objective))) 

Problem:
MINIMIZE
55.16*x104 + 141.69*x11 + 236.24*x15 + 125.17*x21 + 32.31*x31 + 70.67*x41 + 207.61*x51 + 156.55*x54 + 3.97*x62 + 13.31*x66 + 317.19*x72 + 19.69*x73 + 44.23*x77 + 66.51*x81 + 25.83*x93 + 22.95*x98 + 0.0
SUBJECT TO
_C1: S1 - x11 - x21 - x31 - x41 - x51 - x81 = 0

_C2: S2 - x62 - x72 = 0

_C3: S3 - x73 - x93 = 0

_C4: S4 - x104 - x54 = 0

_C5: S5 - x15 = 0

_C6: S6 - x66 = 0

_C7: S7 - x77 = 0

_C8: S8 - x98 = 0

_C9: D1 - x11 - x15 = 0

_C10: D2 - x21 = 0

_C11: D3 - x31 = 0

_C12: D4 - x41 = 0

_C13: D5 - x51 - x54 = 0

_C14: D6 - x62 - x66 = 0

_C15: D7 - x72 - x73 - x77 = 0

_C16: D8 - x81 = 0

_C17: D9 - x93 - x98 = 0

_C18: - D1 - D10 - D2 - D3 - D4 - D5 - D6 - D7 - D8 - D9 + S1 + S2 + S3 + S4
 + S5 + S6 + S7 + S8 = 0

_C19: S1 <= 2365200000

_C20: S2 <= 3153600000

_C21: S3 <= 21759840000

_C22: S4 <= 4730400000

_C23: S5 <= 9460800000

_C24: S6 <= 4730400000

_C25: S7 <= 1504267200

_C26: S8 <= 3784320000

_C27: D1 >= 1129556342.88

_C28: D2 >= 318863386.8

_C29:

For better visualization, we can draw the table which shows how much power each city gets from each powerplant:

In [17]:
result = np.zeros((10,8))
for i in range(10):
    for j in range(8):
        result[i][j] = globals()["x{}{}".format(i+1,j+1)].varValue
        print(result[i][j])
print(result)

0.0
nan
nan
nan
0.0
nan
nan
nan
0.0
nan
nan
nan
nan
nan
nan
nan
0.0
nan
nan
nan
nan
nan
nan
nan
0.0
nan
nan
nan
nan
nan
nan
nan
0.0
nan
nan
0.0
nan
nan
nan
nan
nan
0.0
nan
nan
nan
0.0
nan
nan
nan
36900597000.0
21759840000.0
nan
nan
nan
1504267200.0
nan
0.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.0
nan
nan
nan
nan
0.0
nan
nan
nan
0.0
nan
nan
nan
nan
[[0.0000000e+00           nan           nan           nan 0.0000000e+00
            nan           nan           nan]
 [0.0000000e+00           nan           nan           nan           nan
            nan           nan           nan]
 [0.0000000e+00           nan           nan           nan           nan
            nan           nan           nan]
 [0.0000000e+00           nan           nan           nan           nan
            nan           nan           nan]
 [0.0000000e+00           nan           nan 0.0000000e+00           nan
            nan           nan           nan]
 [          nan 0.0000000e+00           nan           nan         